In [1]:
from dateutil import parser
import numpy as np 
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from pandas.tseries.frequencies import to_offset
from datetime import datetime as dt


In [2]:
dateparse = lambda dates: parser.parse(dates)
data = pd.read_csv('./BTC2.csv', index_col='Date', parse_dates=['Date'], date_parser=dateparse)
f=data.drop(['Change %'],axis=1)
f.head()

,Price,Open,High,Low,Vol.
Date,,,,,
2021-12-31,"46,219.5","47,123.3","48,553.9","45,693.6",58.18K
2021-12-30,"47,123.3","46,470.7","47,901.4","46,003.0",60.96K
2021-12-29,"46,461.7","47,548.4","48,121.7","46,127.8",63.92K
2021-12-28,"47,545.2","50,703.4","50,703.8","47,345.7",74.39K
2021-12-27,"50,678.2","50,783.6","52,016.3","50,459.0",43.90K


In [3]:
f['Vol.']=f['Vol.'].replace({'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(int)
f['Price']=f['Price'].replace({',':''},regex=True).astype(float)
f['Open']=f['Open'].replace({',':''},regex=True).astype(float)
f['High']=f['High'].replace({',':''},regex=True).astype(float)
f['Low']=f['Low'].replace({',':''},regex=True).astype(float)



In [4]:
from datetime import datetime
moon_phases=pd.read_csv('./Dataset/moon_phases.csv')
moon_phases['Date']=moon_phases['Date'].astype(str)+','+moon_phases['Year'].astype(str)
moon_phases.drop(['Year','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)
moon_phases.dropna(inplace=True)
moon_phases['Date']=moon_phases['Date'].apply(lambda x: datetime.strptime(x, "%B %d, %a,%Y"))
df=moon_phases.set_index('Date')
df=df.loc[df['Phase'].isin(['New Moon','Full Moon'])]
df=df.loc[df.index<'2022-01-01']


In [5]:


def take_first(array_like):
    return array_like[0]

def take_last(array_like):
    return array_like[-1]

logic={'Open': take_first, 
                         'High': 'max',
                         'Low': 'min',
                         'Price': take_last,
                         'Vol.': 'sum'}
weekly = f.resample('W').apply(logic)                                 # Weekly resample
                      # to put the labels to Monday , loffset=pd.offsets.timedelta(days=-6)
weekly.index -= to_offset("6D")
weekly

,Open,High,Low,Price,Vol.
Date,,,,,
2019-12-30,7196.4,7501.0,6884.1,7372.5,3141399
2020-01-06,7372.6,8436.4,7366.7,8187.6,6195220
2020-01-13,8188.3,9183.7,8068.4,8706.2,6405840
2020-01-20,8705.7,8805.4,8242.6,8607.8,3900350
2020-01-27,8608.1,9569.0,8585.6,9334.9,4950900
...,...,...,...,...,...
2021-11-29,57288.1,59174.4,42587.8,49405.5,583230
2021-12-06,49412.1,51918.6,46834.0,50081.0,413840
2021-12-13,50075.9,50196.1,45513.4,46704.5,419870


In [6]:
merge=pd.merge(df,f, how='inner', left_index=True, right_index=True)
nm=merge[merge['Phase']=='New Moon']
fm=merge[merge['Phase']=='Full Moon']
fullmoon=fm.copy()
newmoon=nm.copy()
fullmoon['Price']=fullmoon['Price'].apply(lambda x:x-10000)
newmoon['Price']=newmoon['Price'].apply(lambda x:x+10000)
fullmoon

,Phase,Price,Open,High,Low,Vol.
Date,,,,,,
2021-01-17,Full Moon,25839.6,36020.1,36801.3,33883.5,126820
2021-02-16,Full Moon,39169.7,47934.2,50515.8,47044.4,141370
2021-03-18,Full Moon,47656.0,58911.8,60088.0,57099.6,102620
2021-04-16,Full Moon,51379.7,63211.6,63518.5,60027.3,136850
2021-05-16,Full Moon,36426.4,46729.3,49764.3,43920.8,180070
2021-06-14,Full Moon,30529.4,39024.1,40970.4,38769.9,157350
2021-07-13,Full Moon,22728.1,33106.1,33307.7,32259.5,60040
2021-08-11,Full Moon,35564.3,45593.2,46736.5,45367.9,75430
2021-09-10,Full Moon,34842.8,46385.9,47021.6,44283.6,75620


In [7]:
weekly['MA5'] = weekly['Price'].rolling(5).mean()
weekly['MA10'] = weekly['Price'].rolling(10).mean()


In [14]:
import plotly.graph_objects as go


fig = go.Figure(data=[go.Candlestick(x=weekly.index,
                open=weekly['Open'],
                high=weekly['High'],
                low=weekly['Low'],
                close=weekly['Price']),
                      go.Scatter(x=weekly.index, y=weekly.MA5, line=dict(color='orange', width=1)),
                      go.Scatter(x=weekly.index, y=weekly.MA10, line=dict(color='green', width=1)),
                      go.Scatter(x=nm.index,y=nm['Price'],mode='markers',marker=dict(size=15,opacity=0.5,color='red'),name='New Moon'),
                      go.Scatter(x=fm.index,y=fm['Price'],mode='markers',marker=dict(size=15,color='green',opacity=0.5),name='Full Moon')
                     ])

fig.show()

In [9]:
import plotly.graph_objects as go


fig = go.Figure(data=[go.Candlestick(x=f.index,
                open=f['Open'],
                high=f['High'],
                low=f['Low'],
                close=f['Price']),
                      
                      go.Scatter(x=newmoon.index,y=newmoon.Price,mode='markers',marker=dict(size=15),name='New Moon'),
                      go.Scatter(x=fullmoon.index,y=fullmoon.Price,mode='markers',marker=dict(size=15),name='Full Moon')
                     ])

fig.show()

In [10]:
monthly=f.resample('M').apply(logic)
monthly.head()

,Open,High,Low,Price,Vol.
Date,,,,,
2020-01-31,7196.4,9569.0,6884.1,9349.1,23451919
2020-02-29,9349.3,10482.6,8451.9,8543.7,23387210
2020-03-31,8543.8,9180.8,3869.5,6412.5,47841320
2020-04-30,6412.4,9437.5,6157.4,8629.0,39622290
2020-05-31,8628.6,10033.0,8235.6,9454.8,38384270


In [11]:
fig = go.Figure(data=[go.Candlestick(x=monthly.index,
                open=monthly['Open'],
                high=monthly['High'],
                low=monthly['Low'],
                close=monthly['Price'])])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [12]:
def actual(x):
    if x['Change']>0:
        return 1
    else:
        return -1
def pred(x):
    if x['Phase']=='New Moon':
        return 1
    else: 
        return -1
moons=merge[['Phase','Price']].copy()
moons['Change']=moons['Price'].pct_change().shift(-1)
#moons['Change']=moons['Change'].shift(-1)
moons
moons['Actual']=moons.apply(lambda x:actual(x),axis=1)
moons['Pred']=moons.apply(lambda x:pred(x),axis=1)

#nms = moons.apply(lambda x: True if x['Change'] < 0 and x['Phase']=='Full Moon' else False , axis=1)



In [13]:
from sklearn.metrics import classification_report
y_true=moons['Actual']
y_pred=moons['Pred']
print(classification_report(y_true,y_pred))

              precision    recall  f1-score   support

          -1       0.58      0.58      0.58        12
           1       0.62      0.62      0.62        13

    accuracy                           0.60        25
   macro avg       0.60      0.60      0.60        25
weighted avg       0.60      0.60      0.60        25

